In [107]:
import psycopg2
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyecharts
import urllib3,time,csv,datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.finance as mpf
import matplotlib.dates as mpd
import plotly
import plotly.offline as py_offline
import plotly.graph_objs as go
from IPython.display import clear_output

%matplotlib inline

In [108]:
# 在这里把需要优化的双均线策略中要plot的线都在这里计算完成：
# 在这里不要相对和，只要相对差就可以：
"""
T1 = (Data0.Close-Data0.Close[1]) / (Data0.Close[1]) * 100;     // Data0涨幅
T2 = (Data1.Close-Data1.Close[1]) / (Data1.Close[1]) * 100;     // Data1涨幅
TM = T1*(1-N4/100) - T2*(N4/100);                               // 相对差 判断强弱
TMSUM = TMSUM + TM;                                             // 相对差累加
"""
# 首先看有多少个parameters：
N1 = 5
N2 = 50
N3 = 50
N4 = 50
N5 = 2
N6 = 1
ATRLength = 26

In [109]:
# Load csv file and preprocess: 如果直接从SQL读数据，这一个函数可以忽略不用
def preprocess(data_location, file):
    df = pd.read_csv(data_location + file)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df.index = pd.DatetimeIndex(df['time'])
    return df

# 直接从SQL数据库生成资产价格dataframe
def two_assts_from_sql(asset1, asset2, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
    asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
    df1 = pd.read_sql(asset1, con=conn)
    df2 = pd.read_sql(asset2, con=conn)
    conn.close()
    return df1, df2

# 转变数据周期：T/5T/15T/30T/H/2H/4H/D/W/M
def resample(df, period):
    convrted_df = df.resample(period).last()
    convrted_df['open'] = df['open'].resample(period).first()
    convrted_df['high'] = df['high'].resample(period).max()
    convrted_df['low'] = df['low'].resample(period).min()
    convrted_df['close'] = df['close'].resample(period).last()
    convrted_df['volume'] = df['volume'].resample(period).sum()
    # Keep rows with at least 5 non-NaN values
    convrted_df.dropna(thresh=5, inplace=True)
    convrted_df.index = convrted_df['time']
    convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
    return convrted_df

# 为能够plot蜡烛图做timestamps处理的准备：
def cnvrt_date(convrted_df):
    cnvrted_date_df = convrted_df.copy()
    cnvrted_date_df['date'] = mpd.date2num(cnvrted_date_df['time'].dt.to_pydatetime())
    return cnvrted_date_df

# 计算两个资产的相对差累计值：
def two_assets_tmsum(cnvrted_date_df1, cnvrted_date_df2, N4):
    cnvrted_date_df1['close_shifted'] = cnvrted_date_df1['close'].shift(1)
    cnvrted_date_df2['close_shifted'] = cnvrted_date_df2['close'].shift(1)
    T1 = cnvrted_date_df1['close'].diff()/cnvrted_date_df1['close_shifted']
    T2 = cnvrted_date_df2['close'].diff()/cnvrted_date_df2['close_shifted']
    TM = T1*(1-N4/100) - T2*(N4/100)
    tmsum_sr = TM.cumsum()
    return tmsum_sr
    
# Plot TMSUM图线：
def plot_tmsum(tmsum_sr):
#     py_offline.init_notebook_mode()
    tmsum_df = pd.DataFrame(tmsum_sr, columns=['tmsum'])
    tmsum_df = go.Scatter(x = tmsum_df.index,
                          y = tmsum_df['tmsum'])
    data = [tmsum_df]
    py_offline.iplot(data, filename='TMSUM')
    
# Plot 资产实时图
def plot_candlestick(cnvrted_date_df):
#     py_offline.init_notebook_mode()
    candle_df = go.Candlestick(x = cnvrted_date_df.index,
                               open = cnvrted_date_df['open'],
                               high = cnvrted_date_df['high'],
                               low  = cnvrted_date_df['low'],
                               close = cnvrted_date_df['close'])
    data = [candle_df]
    py_offline.iplot(data, filename='Candle Stick', image_width=2, image_height=4)

In [123]:
plot_candlestick(cnvrted_date_df_bch)

In [74]:
# # Improve the plotly function further while keeping the original version in previous cells:
# def plot_candlestick(cnvrted_date_df):
# #     py_offline.init_notebook_mode()
# #     layout = go.Layout(
# #                         autosize=True,
# #                         width=20,
# #                         height=10)
# #     candle_df = go.Candlestick(x = cnvrted_date_df.index,
# #                                open = cnvrted_date_df['open'],
# #                                high = cnvrted_date_df['high'],
# #                                low  = cnvrted_date_df['low'],
# #                                close = cnvrted_date_df['close'])

#     fig = plotly.figurefactory.create_candlestick(
#                                 cnvrted_date_df['open'], 
#                                 cnvrted_date_df['high'], 
#                                 cnvrted_date_df['low'], 
#                                 cnvrted_date_df['close'],
#                                 dates=cnvrted_date_df.index)
#     py.iplot(fig, filename='finance/tcs-candlestick', validate=False)
# #     tmsum = pd.DataFrame(tmsum_sr, columns=['tmsum_sr'])['tmsum_sr'] 
#     # Here we then calculate moving additional plotting lines, e.g. moving averages etc.
    
# #     data = [candle_df]
# #     py_offline.iplot(data, filename='Candle Stick')

# #     fig = go.Figure(data=candle_df, layout=layout)
# #     py.iplot(fig, filename='automargin')

In [ ]:
# while True:
#     # Period的频率不能低于SQL里的数据频率。
#     period = '5T'
#     conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
#     bchz18_1m = """ SELECT time, open, high, low, close, volume from bchz18_1m order by id desc limit 200"""
#     ethz18_1m = """ SELECT time, open, high, low, close, volume from bchz18_1m order by id desc limit 200"""
#     bch_df = pd.read_sql(bchz18_1m, con=conn)
#     eth_df = pd.read_sql(ethz18_1m, con=conn)
#     conn.close()
    
# #     df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
#     bch_df.index = pd.DatetimeIndex(bch_df['time'])
#     eth_df.index = pd.DatetimeIndex(eth_df['time'])
#     res_bch = resample(bch_df, period)
#     res_eth = resample(eth_df, period)
    
#     cnvrted_date_df_bch = cnvrt_date(res_bch)
#     cnvrted_date_df_eth = cnvrt_date(res_eth)
    
#     plot_candlestick(cnvrted_date_df_bch)
#     plot_candlestick(cnvrted_date_df_eth)
#     time.sleep(5)
#     clear_output()

In [120]:
df1_, df2_ = two_assts_from_sql("bchz18_1m", "ethz18_1m", 400)

In [121]:
df2_

,time,open,high,low,close,volume
0,2018-09-21 18:47:00,0.03418,0.03418,0.03418,0.03418,0.0
1,2018-09-21 18:48:00,0.03418,0.03418,0.03418,0.03418,0.0
2,2018-09-21 18:49:00,0.03418,0.03418,0.03418,0.03418,0.0
3,2018-09-21 18:50:00,0.03418,0.03418,0.03418,0.03418,0.0
4,2018-09-21 18:51:00,0.03418,0.03418,0.03418,0.03418,0.0
5,2018-09-21 18:52:00,0.03418,0.03418,0.03418,0.03418,0.0
6,2018-09-21 18:53:00,0.03418,0.03418,0.03418,0.03418,0.0
7,2018-09-21 18:54:00,0.03418,0.03418,0.03418,0.03418,5.0
8,2018-09-21 18:55:00,0.03418,0.03418,0.03418,0.03418,0.0
9,2018-09-21 18:56:00,0.03418,0.03418,0.03418,0.03418,0.0


In [110]:
period = '5T'
conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
bchz18_1m = """ SELECT time, open, high, low, close, volume from bchz18_1m order by id desc limit 800"""
ethz18_1m = """ SELECT time, open, high, low, close, volume from ethz18_1m order by id desc limit 800"""
bch_df = pd.read_sql(bchz18_1m, con=conn)
eth_df = pd.read_sql(ethz18_1m, con=conn)
conn.close()

# 转变时间周期
bch_df.index = pd.DatetimeIndex(bch_df['time'])
eth_df.index = pd.DatetimeIndex(eth_df['time'])
res_bch = resample(bch_df, period)
res_eth = resample(eth_df, period)

# 处理时间column以此方便后面的作图
cnvrted_date_df_bch = cnvrt_date(res_bch)
cnvrted_date_df_eth = cnvrt_date(res_eth)

# plot
plot_candlestick(cnvrted_date_df_eth)
plot_candlestick(cnvrted_date_df_bch)
tmsum_sr = two_assets_tmsum(cnvrted_date_df_bch, cnvrted_date_df_eth, N4)
plot_tmsum(tmsum_sr)
# plot_candlestick(cnvrted_date_df_eth, tmsum_sr)

In [12]:
# Testing if tmsum is calculated correctly.
# tmsum = two_assets_tmsum(cnvrted_date_df_bch, cnvrted_date_df_eth, N4)

In [ ]:
# 用于参考：
# while True:
#     # Period的频率不能低于SQL里的数据频率。
#     period = '5T'
#     conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
#     sqllasttime = """ SELECT time, open, high, low, close, volume from xbtusd_1m order by id desc limit 200"""
#     df = pd.read_sql(sqllasttime, con=conn)
#     conn.close()
    
#     df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
#     df.index = pd.DatetimeIndex(df['time'])
#     res = resample(df, period)
#     cnvrted_date_df = cnvrt_date(res)

#     plot_candlestick(cnvrted_date_df)
#     time.sleep(5)
#     clear_output()

In [122]:
plot_tmsum(tmsum_sr)